# Resumen

<p style="text-align: justify">Los ejemplos constaran de uso de librerias de calculo numerico, estadistico y de visualizacion, se anexo la graficacion en 3D utilizando Matplotlib y manejo de matematica simbolica con sympy, para un ejemplo mas complejo se realizara la simulacion de un sistema de control en lazo cerrado utilizando ecuaciones de espacio de estado y resolviendo por el metodo numerico iterativo de Runge-kutta de orden 4, adiconalmente, el analisis de un sensor de ultrasonido para medir distancia</p>

**Autor: Kleiver Carrasco - kleiver615@gmail.com**

## Librerias

<p style="text-align: justify">Lo primero sera importar las librerias, para importar una libreria se utiliza la palabra reservada import, para hacer el uso de las librerias mas facil se les puede asignar un nombre arbitrario y mas corto.</p>

In [ ]:
import scipy as sp

<p style="text-align: justify">Con esto hemos importado la libreria scipy y le hemos asignado el nombre de sp, para usarla basta con escribir sp y un punto para acceder a sus atributos.</p>

[Documentacion de Scipy](https://docs.scipy.org/doc/scipy/reference/index.html)

In [ ]:
sp.pi

<p style="text-align: justify">A continuacion vamos a exportar las librerias que se utilizaran durante el resto ejemplos, es costumbre importar todas las librerias primero y en la parte superior del documento.</p>

In [ ]:
# Graficas estaticas en el notebook
%matplotlib inline

# Graficas interactivas en una ventana aparte
# %matplotlib qt 

# Graficas interactivas, requiere Node.js y la extension de jupyterlab correspondiente
# %matplotlib widget 

from matplotlib import pyplot as plt # Graficacion en 2D
from mpl_toolkits.mplot3d import Axes3D # Graficacion en 3D
from tqdm import tqdm # Observar progreso de un ciclo
import control as ctrl # Libreria de control de procesos
import numpy as np # Libreria de calculo numerico n dimensional
plt.close('all') # Para limpiar las figuras residuales

# Inicio de los ejemplos

## Ejemplo de graficacion en 3D

<p style="text-align: justify">Utilizando Matplotlib se pueden realizar graficas en 3D, en el particular caso de los jupyter notebooks, las graficas siempre son estaticas, no obstante, se puede agregar en la importaciones la linea <b>%matplotlib widget</b> en sustitucion de <b>%matplotlib inline</b> la cual permite tener graficas interactivas.</p>

>**Nota:** Las graficas en 3D funcionan con <b>%matplotlib inline</b>, pero seran imagenes estaticas,  para volverlas interactivas se debe:
1. Descargar e instalar Node.js desde: [Node.js link](https://nodejs.org/es/download/)
2. Abrir una consola de comandos (cmd) y ejecutar los siguientes comandos:
    1. jupyter labextension install @jupyter-widgets/jupyterlab-manager
    2. jupyter labextension install jupyter-matplotlib <br>

><p style="text-align: justify">Una vez hecho esto, se puede inicializar jupyterlab como de costumbre y realizar la sustitucion de <b>%matplotlib inline</b> por <b>%matplotlib widget</b> , adicionalmente, si jupyterlab se encuentra ya en ejecucion, es recomendable realizar un reinicio del kernel y volver a ejecutar todas las celdas.</p>
><p style="text-align: justify"> Otra opcion es cambiar <b>%matplotlib inline</b> por <b>%matplotlib qt</b> el cual provocara que las graficas ya no se vean en el notebook sino en una ventana aparte, esta ventana sera interactiva. </p>

[Galeria de ejemplos de Matplotlib](https://matplotlib.org/gallery/index.html)

Como ejemplo se graficara:
$$
\begin{align}
    z = \sin (x) + \cos (y) \quad con\quad x,y \in [-5, 5]
\end{align}
$$

In [ ]:
x_samp = np.linspace(-5, 5, 20)
y_samp = np.linspace(-5, 5, 20)
x, y = np.meshgrid(x_samp, y_samp)

z = np.sin(x) + np.cos(y)

fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot(111, projection='3d')
plt.title('Grafica en 3D')
surf = ax.plot_surface(x, y, z, cmap='viridis')

## Matematica simbolica

Para realizar matematica simbolica se utiliza la libreria sympy, en la siguiente celda vamos a importar la libreria para mostrar que no es necesario realizar siempre la importacion en el tope del notebook.

[Documentacion de sympy](https://docs.sympy.org/latest/index.html)

In [ ]:
import sympy
from IPython.display import display

x, f, a, b = sympy.symbols('x f(x) a b')

# Funcion a integrar
print('Funcion a integrar')
display(sympy.Eq(f, x**2 * sympy.exp(x) * sympy.cos(x)))
display(sympy.Integral(f, (x, a, b)))
display(sympy.Integral(x**2 * sympy.exp(x) * sympy.cos(x), (x, a, b)))

print('Resultado simbolico')
display(sympy.integrate(x**2 * sympy.exp(x) * sympy.cos(x), (x, a, b)))

In [ ]:
# Se puede obtener el resultado numerico si sustituimos a y b por valores
print('Resultado numerico')
display(sympy.integrate(x**2 * sympy.exp(x) * sympy.cos(x), (x, 0, 1)))

In [ ]:
# Otro modo es obtener el resultado simbolico y luego si realizar la sustitucio de a y b
inte_result = sympy.integrate(x**2 * sympy.exp(x) * sympy.cos(x), (x, a, b))
print('Resultado numerico alterno')
display(inte_result.evalf(subs={a:0, b:1}))

## Ecuaciones de espacio de estado

<p style="text-align: justify">En ingeniería de control, una representación de espacios de estados es un modelo matemático de un sistema físico descrito mediante un conjunto de entradas, salidas y variables de estado relacionadas por ecuaciones diferenciales de primer orden. Su forma generica viene dada por:</p>

$$
\begin{align}
    \dot{x}(t) &= Ax(t) + Bu(t) \tag{1} \\
    y(t) &= Cx(t) + Du(t) \tag{2}
\end{align}
$$

Representando x, y, u, A, B, C y D en sus formas de matriz queda como sigue:

$$
\begin{bmatrix}
\dot{x}_{1}\\
\dot{x}_{2}\\
\vdots\\
\dot{x}_{n}\\
\end{bmatrix}=
\begin{bmatrix}
a_{11} & a_{12} & \cdots & a_{1n}\\
a_{21} & a_{22} & \cdots & a_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
a_{n1} & a_{n2} & \cdots & a_{nn}\\
\end{bmatrix}
\begin{bmatrix}
x_{1}\\
x_{2}\\
\vdots\\
x_{n}\\
\end{bmatrix}+
\begin{bmatrix}
b_{11} & b_{12} & \cdots & b_{1n}\\
b_{21} & b_{22} & \cdots & b_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
b_{n1} & b_{n2} & \cdots & b_{nn}\\
\end{bmatrix}.
\begin{bmatrix}
u_{1}\\
u_{2}\\
\vdots\\
u_{n}\\
\end{bmatrix} \tag{3}
$$

$$
\begin{bmatrix}
y_{1}\\
y_{2}\\
\vdots\\
y_{n}\\
\end{bmatrix}=
\begin{bmatrix}
c_{11} & c_{12} & \cdots & c_{1n}\\
c_{21} & c_{22} & \cdots & c_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
c_{n1} & c_{n2} & \cdots & c_{nn}\\
\end{bmatrix}
\begin{bmatrix}
x_{1}\\
x_{2}\\
\vdots\\
x_{n}\\
\end{bmatrix}+
\begin{bmatrix}
d_{11} & d_{12} & \cdots & d_{1n}\\
d_{21} & d_{22} & \cdots & d_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
d_{n1} & d_{n2} & \cdots & d_{nn}\\
\end{bmatrix}.
\begin{bmatrix}
u_{1}\\
u_{2}\\
\vdots\\
u_{n}\\
\end{bmatrix} \tag{4}
$$

### Metodo de Runge-Kutta de 4th orden

<p style="text-align: justify">El metodo de Runge-Kutta es un metodo de resolucion de sistemas de ecuaciones diferenciales ordinarias de primer orden de forma iterativa, es decir, de forma numerica y aproximada, el metodo se puede aplicar con los siguientes calculos:</p>

$$
\begin{align}
k_1^i &= h\cdot f_i(t_n,\ x^{(n)}) \tag{5}\\
k_2^i &= h\cdot f_i(t_n + \frac{h}{2},\ x^{(n)} + \frac{k_1}{2}) \tag{6}\\
k_3^i &= h\cdot f_i(t_n + \frac{h}{2},\ x^{(n)} + \frac{k_2}{2}) \tag{7}\\
k_4^i &= h\cdot f_i(t_n + h,\ x^{(n)} + k_3) \tag{8}
\end{align}
$$

<p style="text-align: justify">Donde h es el tamaño de paso, al final de cada iteracion se debe calcular el siguiente valor de x:</p>

$$
x^{(n+1)} = x^{(n)} + \frac{1}{6} (k_1 + 2k_2 + 2k_3 + k_4)\tag{9}
$$

<p style="text-align: justify">Luego basta con sustituir todos los $x^{(n)}$ en la ecuacion $y(t) = Cx(t) + Du(t)$ para obtener los valores de salida. El siguiente codigo es una forma de resolver ecuaciones diferenciales de forma iterativa utilizando Runge-Kutta:</p>

In [ ]:
def runge_kutta(t, ss, h, x, u):
    k1 = h * (ss.A * x + ss.B * u)
    k2 = h * (ss.A * (x+k1/2) + ss.B * u)
    k3 = h * (ss.A * (x+k2/2) + ss.B * u)
    k4 = h * (ss.A * (x+k3) + ss.B * u)
    x = x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
    y = ss.C * x + ss.D * u
    return y, x

### Controlador PID

<p style="text-align: justify">Para demostrar el uso del metodo de Runge-Kutta, se ejecutara una prueba escalon a un proceso, para esto, vamos a realizar un lazo cerrado junto con un controlador PID, un PID puede ser representado en el dominio del tiempo con la siguiente ecuacion:</p>

\begin{equation}
sc(t) = K_{p}e(t)+  K_{i}\int_{0}^{t}e(\tau) d\tau + K_{d} \frac{d}{dt}e(t) \tag{10}
\end{equation}

El codigo que defino al controlador es el siguiente:

In [ ]:
def PID(vm, set_point, ts, s_integral, error_anterior):
    error = set_point - vm
    error_anterior = error
    s_proporcional = error
    s_integral = s_integral + error * ts
    s_derivativa = (error - error_anterior) / ts
    s_control = s_proporcional * kp + s_integral * ki + s_derivativa * kd
    return s_control, s_integral, error_anterior

#### Prueba en lazo abierto

<p style="text-align: justify">Antes de pasar a realizar el calculo para lazo cerrado, vamos a utilizar la libreria de control para realizar la prueba escalon en lazo abierto y compararla con la version RK. El proceso que vamos a utilizar viene dado por la siguiente funcion de transferencia:</p>

\begin{equation}\label{eq:pidtiempo}
H(s) = \frac{s + 2}{s^2 + 2s + 5} \tag{11}
\end{equation}


In [ ]:
# Definicion del proceso
proceso1 = ctrl.tf([1, 2], [1, 2, 5])

# Con libreria de control
tiempo_a, out_a = ctrl.step_response(proceso1)

# Con RK
proceso1_rk = ctrl.tf2ss(proceso1)
x0 = np.zeros_like(proceso1_rk.B)
h_p = 0.01

# Para coordinar tiempos se utilizara el vector de tiempo generado por la libreria de control
rk_tiempo = np.linspace(0, max(tiempo_a), int(max(tiempo_a)/h_p) + 1)
rk_out = [0]

for i in rk_tiempo[1:]:
    temp, x0 = runge_kutta(i, proceso1_rk, h_p, x0, 1)
    rk_out.append(np.asscalar(temp[0]))

plt.figure(figsize=(9,5))
plt.plot(tiempo_a, out_a, 'r', linewidth=1.5, label='Original')
plt.plot(rk_tiempo, rk_out, linestyle=':', linewidth=4, label='RK', dashes=(1, 2))
plt.title('Respuesta escalon en lazo abierto')
plt.legend()
plt.grid()

### Prueba en lazo cerrado

<p style="text-align: justify">Lo siguiente sera crear de forma manual un ciclo que recreara el lazo cerrado de control, se inicializaran las variables que se consideren. Finalmente se grafican los resultados obtenidos y se comparan con la respuesta dada por la libreria de control. Se debe aclarar que aunque se utilizara un controlador PID, la unica ganancia activa sera Kp y esta estara en un valor igual a 1, esto es igual a que si el PID no existiera, esto se realizo de este modo para mantener la prueba simple.</p>

><p style="text-align: justify"><b>Nota:</b> El codigo del PID es 100% funcional, por tanto, si se asignan otros valores a kp, ki, y kd se pueden ver los efectos del controlador en accion.</p>

In [ ]:
proceso = ctrl.tf([1, 2], [1, 2, 5])

# Con libreria de control
feedbackp = ctrl.feedback(proceso)
tiempo_c, out_c = ctrl.step_response(feedbackp)

# Con RK
proceso = ctrl.tf2ss(proceso)
dt = 0.01

# Para coordinar tiempos se utilizara el vector de tiempo generado por la libreria de control
rk_tiempoc = np.linspace(0, max(tiempo_c), int(max(tiempo_c)/dt) + 1) # Tiempo de simulacion

x0 = np.zeros_like(proceso.B)
Vp = 0
error_a = 0
si_t = 0

salida = [0]
señal_g = [0]

# Ganancias del PID
kp = 1
ki = 0
kd = 0

for i in tqdm(rk_tiempoc[1:]):
    sc_t, si_t, error_a = PID(Vp, 1, dt, si_t, error_a)
    y_t, x_t = runge_kutta(i, proceso, dt, x0, sc_t)
    Vp = np.asscalar(y_t[0])
    x0 = x_t
    señal_g.append(sc_t)
    salida.append(Vp)

plt.figure(figsize=(9,5))
plt.plot(tiempo_c, out_c, 'r', linewidth=1.5, label='Original')
plt.plot(rk_tiempoc, salida, linestyle=':', linewidth=4, label='RK', dashes=(1, 2))
plt.title('Respuesta escalon en lazo cerrado')
plt.legend()
plt.grid()

### Utilidad

<p style="text-align: justify">Entonces, ¿Para que utilizar el metodo de Runge-Kutta si el codigo por libreria es mucho mas corto? La respuesta es simple, la libreria de control solo es compatible con funciones de transferencia y ecuaciones de espacio de estado, si se quisiera simular un controlador difuso, no habria modo de simularlo utilizando la libreria de control, por el contrario, para el caso por Runge-Kutta acoplar un controlador difuso seria relativamente simple, bastaria con sustituir la funcion del PID por la correspondiente al controlador difuso. Adicionalmente, si se quiere simular muestra a muestra la libreria de control no seria apta, mientras que Runge-Kutta seria ideal por ser un metodo de resolucion iterativo.</p>

## Sensor de ultrasonido

### Principio de funcionamiento

<p style="text-align: justify">Los dispositivos de ultrasonido son usados en muchos campos de medida, particularmente para medir flujo, nivel y desplazamiento. El ultrasonido es una banda de frecuencias que se encuentra por arriba de los 20 kHz, esto es, por arriba del rango de frecuencias audibles para el humano. Los dispositivos de medida que hacen uso del ultrasonido consisten de un emisor de ondas de ultrasonido y un receptor de la onda, la variable medida es determinada midiendo el cambio en el tiempo que le toma a la onda viajar entre el emisor y el receptor.</p>

<p style="text-align: justify">La forma más común de sensor de ultrasonido es usando un cristal piezoeléctrico contenido en un encapsulado. Este elemento puede operar como emisor y como receptor, están disponibles para operar con frecuencias que varían entre 20 kHz y 15 MHz.La velocidad de transmisión del ultrasonido cambia en función del medio por el que se propaga, cuando se propaga por el aire, la velocidad del ultrasonido es afectada por factores ambientales tales como temperatura, humedad y turbulencia en el aire. Entre los factores que afectan al ultrasonido, la temperatura tiene el mayor efecto. La velocidad del ultrasonido varia con la temperatura de acuerdo a:</p>

\begin{equation}\label{eq:Tsound}
V = 331.6+0.6T\ m/s \tag{12}
\end{equation}

#### Datos tecnicos del sensor

- Modelo: LV-MaxSonar-EZ1
- Voltaje de alimentación: (5.5 - 2.5) $\left[ V\right]$
- Frecuencia de operación: 42 kHz 
- Rango de detección: (254 - 0) $\left[ in\right]$ 
- Rango útil: (254 - 6) $\left[ in\right]$
- Resolución: 1 $\left[ in\right]$

<p style="text-align: justify">Las distancias utilizando el sensor fueron tomas desde 20cm hasta 80cm, su salida fue medida con un voltimetro, los datos se encuentras en milivoltios. Se realizara un analisis estadistico del sensor utilizando la libreria de estadisticas pandas.</p>

[Documentacion de Pandas](https://pandas.pydata.org/pandas-docs/stable/)

<p style="text-align: justify">Lo primero sera importar las librerias que no se han cargado aun, como pandas, ademas, cargaremos la data en un DataFrame de pandas, el DataFrame puede ser creado a partir de multiples formatos de archivos como .csv o .xls, en este caso hemos cargado la data manualmente y transformando los milivoltios a cm utilizando la formula del fabricante.</p>

\begin{align}\label{eq:mvin}
d\left[in \right] &= \frac{mV}{1000}\times\frac{512}{Vcc}\left[in \right] \tag{13} \\
d\left[cm \right] &= d\left[in \right]\times\frac{1cm}{0.3937in} \tag{14} \\
\end{align}

<p style="text-align: justify">El voltaje a la hora de la medicion fue de Vcc = 4.88V, este voltaje fue el arrojado por la fuente de voltaje del laboratorio.</p>

In [ ]:
import pandas as pd
import matplotlib.ticker as mticker

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format

n_patrones = 7
n_filas = 8

Data_Calibracion = pd.DataFrame(np.array([[20,30,40,50,60,70,80],
                                          [82.2,119.7,148.5,187.5,226.1,254.2,292.3],
                                          [84,122.8,151.4,189.5,225.3,260.1,295],
                                          [84.3,122.4,151.5,189.4,227.3,262.5,295.1],
                                          [86.1,125.3,154,191,229,261.2,294.9],
                                          [84.5,123.5,152.6,192,231,266,296],
                                          [86,124,152,190,227,264,296],
                                          [85,123,152.2,191.5,230,267,295.6],
                                          [84.2,123,152.5,191,229.5,267.2,296.2]]))

header = Data_Calibracion.iloc[0]
Data_Calibracion = Data_Calibracion[1:]
Data_Calibracion.columns = header

# Llevando milivoltios a cm
Data_Calibracion = Data_Calibracion.divide(1000)
Data_Calibracion = Data_Calibracion.multiply(512/4.88)
Data_Calibracion = Data_Calibracion.divide(0.3937)

<p style="text-align: justify">El comando describe() genera datos estadisticos basicos del DataFrame como media, desviacion estandar y cuartiles para cada columna, ademas, pandas permite calcular muchos otros datos estadisticos.</p>

In [ ]:
Data_Calibracion.describe()

In [ ]:
# Calculo de datos estadisticos

# Creando un nuevo DataFrame
Stats_results = pd.DataFrame()

Stats_results['Media'] = Data_Calibracion.mean()
Stats_results['Desviacion Estandar'] = Data_Calibracion.std()
Stats_results['Desviacion Media'] = Data_Calibracion.mad()
Stats_results['Mediana'] = Data_Calibracion.median()
Stats_results['Varianza'] = Data_Calibracion.var()
Stats_results['Error estandar de la media'] = Data_Calibracion.sem()
Stats_results['Curtosis'] = Data_Calibracion.kurt()

Stats_results.index.name = 'Patrones'
Stats_results.T

<p style="text-align: justify">Pandas permite realizar multiples graficas estadisticas para los DataFrame, en este caso vamos a visualizar los histogramas y una grafica de cajas de bitoges.</p>

In [ ]:
# Histogramas
Data_Calibracion.hist(figsize=(10,6))
plt.suptitle('Histogramas')
plt.subplots_adjust(hspace=0.5)

In [ ]:
# Cajas y bigotes
Estilo_Outliers = dict(marker='o', markerfacecolor='red', markersize=7)
Data_Calibracion.boxplot(flierprops=Estilo_Outliers,figsize=(12,7))
plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f cm'))

### Linealidad

<p style="text-align: justify">Para el calculo de linealidad se construye una gráfica de valor medido vs valor patrón los dos en porcentaje del SPAN. Se trazan dos rectas paralelas que cubra toda la dispersión de puntos de la gráfica.</p>
    
\begin{equation}\label{eq:15}\tag{15}
Lin\left(\%\right) =\pm\frac{1}{2}\times\frac{\left|V_{1}-V_{2}\right|}{SPAN}
\end{equation}

In [ ]:
# Linealidad ------------------------------------------------------------------------
Valores = Data_Calibracion.values
ValorEntrada = np.array([20, 30, 40, 50, 60, 70, 80])
ValorSalida = Valores
span = 60
Span_real = (ValorEntrada-20) * 100 /span

Valor_medido_span = (ValorSalida-20)*100/span
plt.figure(1,figsize=(12,8))
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%d %%'))
plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%d %%'))
ScaSize = 9*[20]

Resta = Valor_medido_span-Span_real
bias_positivo = np.max(Resta)
bias_negativo = np.min(Resta)

# Graficacion y detalles del plot
plt.plot([-10,110], [-10+bias_positivo,110+bias_positivo], linestyle='--',
         label='Recta paralela +', alpha= 0.5, color='darkcyan' )
plt.plot([-10,110], [-10+bias_negativo,110+bias_negativo], linestyle='--',
         label='Recta paralela -', alpha= 0.5, color='darkcyan' )

plt.plot([0,100], [0,100], label='Recta Optima', linewidth=2, alpha=0.8, color='darksalmon')
plt.plot([60,60], [20,100], label='Recta para señalar V1 y V2', linestyle='-.', alpha=0.5, color='darkgreen', linewidth=3)

# labels
plt.annotate('V1', (61,74), arrowprops=dict(arrowstyle='->'), xytext=(65, 87))
plt.annotate('V2', (61,56), arrowprops=dict(arrowstyle='->'), xytext=(65, 47))

# Mediciones Scatter plot
for i in range(n_filas):
    plt.scatter(Span_real, Valor_medido_span[i], color='navy', s = ScaSize)

plt.legend(["Recta paralela +","Recta paralela -", "Recta optima", "Recta para señalar V1 y V2", "Mediciones"])

# Ejes
plt.plot([0,0],[-10,120], label='% Medicion', alpha= 0.4, linewidth=3, color='k', linestyle='--')
plt.annotate('% Medicion', (-12,110))
plt.plot([-10,110],[0,0], label='% Patron', alpha= 0.4, linewidth=3, color='k', linestyle='--')
plt.annotate('% Patron', (100,-10))
plt.grid()
plt.show()

<p style="text-align: justify">Calculando $V_{1}$ y $V_{2}$ de manera gráfica y aplicando (15) con $V_{1}$ y $V_{2}$ se obtuvo:</p>

In [ ]:
# Calculo de linealidad
Linealidad = (1/2)*abs((Span_real+bias_positivo)-(Span_real+bias_negativo)) 

print(" Linealidad : ",Linealidad[0],"%")

### Estimacion con polinomio

Para estimar la respuesta del sensor de ultrasonido se realizara el calculo de un polinomio de grado 7 de la forma:

\begin{equation}\label{eq:Polinomio}
f\left(x \right)=a_{7}x^{7}+a_{6}x^{6}+a_{5}x^{5}+a_{4}x^{4}+a_{3}x^{3}+a_{2}x^{2}+a_{1}x^{1}+a_{0} \tag{16}
\end{equation}

Python se utilizara para obtener los coeficientes $a_7, a_6, a_5, a_4, a_3, a_2, a_1\ y\ a_0$, se compara la respuesta teorica esperada (linea recta) con la respuesta real obtenida (muestras) y la respuesta dada por el polinomio de grado 7.

In [ ]:
# Mediciones
Voltaje = np.asarray([75.3, 84.538, 122.963, 151.838, 190.238, 228.150, 262.775, 295.137, 305.5])

# Valores reales + 2 valores extra en los extremos
Patrones = np.asarray([17, 20.000, 30.000, 40.000, 50.000, 60.000, 70.000, 80.000, 83 ])

z1 = np.polyfit(Voltaje,Patrones,7)
print(f" Coeficientes del polonomio: {z1}")

CentimetrosC = np.poly1d(z1,variable='x')
print(f"\nPolinomio:\n{CentimetrosC}  -->    Con x = Voltaje\n")

plt.figure(1, figsize=(10,7))
plt.title("Comaparacion entre respuesta real, teorica y estimacion")
plt.gca().xaxis.set_major_formatter(mticker.FormatStrFormatter('%d mV'))
plt.gca().yaxis.set_major_formatter(mticker.FormatStrFormatter('%d cm'))

plt.scatter(Voltaje , Patrones)
plt.plot( Voltaje, CentimetrosC(Voltaje))
plt.plot(Voltaje, Voltaje/3.75245)

plt.legend(["Estimacion con polinomio","Respuesta teorica","Respuesta real"])
plt.grid()
# plt.savefig("Estimacion.pdf", bbox_inches='tight', pad_inches=0.1, format='pdf')